# Set up

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GITHUB_TOKEN = user_secrets.get_secret("GITHUB_TOKEN")

In [2]:
BRANCH_NAME = 'feature/gpu-acceleration'
!git clone -b {BRANCH_NAME} https://{GITHUB_TOKEN}@github.com/tadtd/binsketch-algorithm

Cloning into 'binsketch-algorithm'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 248 (delta 145), reused 177 (delta 77), pack-reused 0 (from 0)
Receiving objects: 100% (248/248), 148.19 KiB | 3.02 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [3]:
%cd binsketch-algorithm

/kaggle/working/binsketch-algorithm


# Configuration

In [4]:
import os
import gdown
import zipfile

# Process data

In [5]:
DRIVE_URL = 'https://drive.google.com/drive/folders/1ARBY9cIGj_jigi5Y88CtUy-GMj2clrXj'
RAW_DATA_PATH = './raw'
PROCESSED_DATA_PATH = './data'

In [6]:
def download_dataset(drive_url, target_folder):
    print(f"Processing: {drive_url}")
    
    # Create the folder if it doesn't exist
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    if "drive/folders" in drive_url or "folder" in drive_url:
        print("Downloading individual files directly...")
        gdown.download_folder(drive_url, output=target_folder, quiet=False, use_cookies=False)
        print(f"\n[SUCCESS] Folder contents downloaded to: {target_folder}")
    else:
        print("\n[INFO] Detected a Drive FILE link.")
        zip_path = os.path.join(target_folder, "temp_dataset.zip")
        output = gdown.download(drive_url, zip_path, quiet=False, fuzzy=True)
        
        if not output:
            print("[ERROR] Download failed.")
            return

        print(f"\nUnzipping {output}...")
        try:
            with zipfile.ZipFile(output, 'r') as zip_ref:
                zip_ref.extractall(target_folder)
            print(f"[SUCCESS] Extracted to: {target_folder}")
            
            # Clean up the zip file
            os.remove(output)
            
        except zipfile.BadZipFile:
            print("[ERROR] The downloaded file was not a valid zip file.")
            print("Check if the file on Drive is actually a .zip archive.")

In [7]:
download_dataset(DRIVE_URL, RAW_DATA_PATH)

Processing: https://drive.google.com/drive/folders/1ARBY9cIGj_jigi5Y88CtUy-GMj2clrXj


Retrieving folder contents


Retrieving folder 10Y_7o78v8HhztcE7bzgV3M94N0vXjt_U bbc
Processing file 1lgysq7G_lc_zc71dGGqWoHejDCixcOVr docword.bbc.txt.gz
Processing file 1kO9AOWWuACtNsgmA9yyp9C6pSoB8KW3U vocab.bbc.txt
Retrieving folder 1D9mMF6ealOinLAsmwXsZs9IMNPBqzuM0 enron
Processing file 1JuUxpaQRAl1yZGqb3xcSP8nfK3_1NZ8y docword.enron.txt.gz
Processing file 16Rn70xrTnYOIkVm2mz4ICR2bEbyzyvGQ vocab.enron.txt
Retrieving folder 1YMxNXk2-7Ok1_3gnIguPwWt365C5X7Et kos
Processing file 1c1bJ-eX5Rp729zGSeGGXqwyor4DfWsrx docword.kos.txt.gz
Processing file 1YL0wnFKLJz-h6emVWHYDfBET5cAZhdKi vocab.kos.txt
Retrieving folder 17JU-ouMBLAUilZiaxE1xzeKfpFvDy9PA nytimes
Processing file 1gsmnfyNEAtA_3kdU5GMhwnUX-vOlc_9A docword.nytimes.txt.gz
Processing file 1jAnAFekn8u-e_FO1tsElhr-gP_dvSXkx vocab.nytimes.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1lgysq7G_lc_zc71dGGqWoHejDCixcOVr
To: /kaggle/working/binsketch-algorithm/raw/bbc/docword.bbc.txt.gz
100%|██████████| 490k/490k [00:00<00:00, 113MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kO9AOWWuACtNsgmA9yyp9C6pSoB8KW3U
To: /kaggle/working/binsketch-algorithm/raw/bbc/vocab.bbc.txt
100%|██████████| 77.2k/77.2k [00:00<00:00, 53.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JuUxpaQRAl1yZGqb3xcSP8nfK3_1NZ8y
To: /kaggle/working/binsketch-algorithm/raw/enron/docword.enron.txt.gz
100%|██████████| 12.3M/12.3M [00:00<00:00, 51.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=16Rn70xrTnYOIkVm2mz4ICR2bEbyzyvGQ
To: /kaggle/working/binsketch-algorithm/raw/enron/vocab.enron.txt
100%|██████████| 236k/236k [00:00<00:00, 83.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1c1bJ-eX5Rp729zGSeGGXqwyor


[SUCCESS] Folder contents downloaded to: ./raw



Download completed


In [8]:
!python convert.py


Processing BBC dataset
Loading vocabulary from raw/bbc/vocab.bbc.txt...
Loading document-word data from raw/bbc/docword.bbc.txt.gz...
  Documents: 2225, Words: 9635, Non-zeros: 158079
Creating sparse matrix...
Reading sparse data: 100%|████| 158079/158079 [00:00<00:00, 332231.65 entries/s]
Converting to DataFrame in batches of 500 documents...
Processing batches: 100%|█████████████████████| 5/5 [00:00<00:00, 84.06 batch/s]
Concatenating batches...
DataFrame shape: (2225, 9635)
Sparsity: 99.26%

BBC DataFrame Preview:
             ad  sale  boost  time  ...  quarterli  media  giant  jump
document_id                         ...                               
0             1     1      1     1  ...          0      1      1     0
1             0     0      1     1  ...          0      0      0     0
2             0     1      0     0  ...          0      0      1     0
3             0     0      0     0  ...          0      0      0     0
4             0     0      0     0  ...          0

# Experiment

## Experiment 1: Accuracy of Estimation

### Experiment on NYTimes to calculate $MSE$ using Inner Product

In [9]:
DATASET = 'nytimes'
SIMILARITY_SCORE = 'inner_product'
METRIC = 'mse'
data_path = f'./data/{DATASET}_binary.npy'
THRESHOLD = [120, 150, 180, 200, 220, 250, 270, 300]
threshold_str = ' '.join([str(t) for t in THRESHOLD])

In [10]:
!python save_ground_truth.py --experiment 1 \
                             --data_path {data_path} \
                             --similarity_score {SIMILARITY_SCORE} \
                             --seed 42 \
                             --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/save_ground_truth.py", line 15, in <module>
    from src import cosine_similarity, jaccard_similarity, inner_product
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


In [11]:
!python main.py --algo BinSketch BCS \
                --data_path {data_path} \
                --ground_truth_path ground_truth_exp1_{DATASET}_{SIMILARITY_SCORE}.json \
                --seed 42 \
                --threshold {threshold_str} \
                --similarity_score {SIMILARITY_SCORE} \
                --eval_metric {METRIC} \
                --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/main.py", line 10, in <module>
    from src.gpu_utils import GPUConfig
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


### Experiments on ENRON to calculate $-\log(MSE)$ using Cosine Similarity

In [12]:
DATASET = 'enron'
SIMILARITY_SCORE = 'cosine_similarity'
METRIC = 'minus_log_mse'
data_path = f'./data/{DATASET}_binary.npy'
THRESHOLD = [.1, .2, .3, .4, .5, .7, .8, .9]
threshold_str = ' '.join([str(t) for t in THRESHOLD])

In [13]:
!python save_ground_truth.py --experiment 1 \
                             --data_path {data_path} \
                             --similarity_score {SIMILARITY_SCORE} \
                             --seed 42 \
                             --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/save_ground_truth.py", line 15, in <module>
    from src import cosine_similarity, jaccard_similarity, inner_product
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


In [14]:
!python main.py --algo BinSketch SimHash MinHash \
                --data_path {data_path} \
                --ground_truth_path ground_truth_exp1_{DATASET}_{SIMILARITY_SCORE}.json \
                --seed 42 \
                --threshold {threshold_str} \
                --similarity_score {SIMILARITY_SCORE} \
                --eval_metric {METRIC} \
                --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/main.py", line 10, in <module>
    from src.gpu_utils import GPUConfig
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


### Experiments on NYTimes to calculate $- \log(MSE)$ using Jaccard Similarity

In [15]:
DATASET = 'nytimes'
SIMILARITY_SCORE = 'jaccard_similarity'
METRIC = 'minus_log_mse'
data_path = f'./data/{DATASET}_binary.npy'
THRESHOLD = [.1, .2, .3, .4, .5, .7, .8, .9]
threshold_str = ' '.join([str(t) for t in THRESHOLD])

In [16]:
!python save_ground_truth.py --experiment 1 \
                             --data_path {data_path} \
                             --similarity_score {SIMILARITY_SCORE} \
                             --seed 42 \
                             --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/save_ground_truth.py", line 15, in <module>
    from src import cosine_similarity, jaccard_similarity, inner_product
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


In [17]:
!python main.py --algo BinSketch BCS MinHash \
                --data_path {data_path} \
                --ground_truth_path ground_truth_exp1_{DATASET}_{SIMILARITY_SCORE}.json \
                --seed 42 \
                --threshold {threshold_str} \
                --similarity_score {SIMILARITY_SCORE} \
                --eval_metric {METRIC} \
                --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/main.py", line 10, in <module>
    from src.gpu_utils import GPUConfig
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


### Experiments on BBC to calculate $-\log(MSE)$ using Jaccard Similarity

In [18]:
DATASET = 'bbc'
SIMILARITY_SCORE = 'jaccard_similarity'
METRIC = 'minus_log_mse'
data_path = f'./data/{DATASET}_binary.npy'
THRESHOLD = [.1, .2, .3, .4, .5, .7, .8, .9]
threshold_str = ' '.join([str(t) for t in THRESHOLD])

In [19]:
!python save_ground_truth.py --experiment 1 \
                             --data_path {data_path} \
                             --similarity_score {SIMILARITY_SCORE} \
                             --seed 42 \
                             --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/save_ground_truth.py", line 15, in <module>
    from src import cosine_similarity, jaccard_similarity, inner_product
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


In [20]:
!python main.py --experiment 1 \
                --algo BinSketch BCS MinHash \
                --data_path {data_path} \
                --ground_truth_path ground_truth_exp1_{DATASET}_{SIMILARITY_SCORE}.json \
                --seed 42 \
                --threshold {threshold_str} \
                --similarity_score {SIMILARITY_SCORE} \
                --eval_metric {METRIC} \
                --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/main.py", line 10, in <module>
    from src.gpu_utils import GPUConfig
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


## Experiment 2: Ranking

### Experiments on ENRON to calculate Accuracy using Jaccard Similarity

In [21]:
THRESHOLD = [.1, .2, .4, .5, .6, .7, .85, .95]
threshold_str = ' '.join([str(t) for t in THRESHOLD])
RETRIEVAL_METRIC = 'accuracy'
DATASET = 'enron'
data_path = f'./data/{DATASET}_binary.npy'
SIMILARITY_SCORE = 'jaccard_similarity'

In [22]:
!python save_ground_truth.py --experiment 2 \
                             --data_path {data_path} \
                             --similarity_score {SIMILARITY_SCORE} \
                             --train_ratio .9 \
                             --seed 42 \
                             --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/save_ground_truth.py", line 15, in <module>
    from src import cosine_similarity, jaccard_similarity, inner_product
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


In [23]:
!python main.py --experiment 2 \
                --algo BinSketch BCS MinHash \
                --data_path {data_path} \
                --ground_truth_path ground_truth_exp2_{DATASET}_{SIMILARITY_SCORE}.json \
                --train_ratio .9 \
                --seed 42 \
                --threshold {threshold_str} \
                --similarity_score {SIMILARITY_SCORE} \
                --retrieval_metric {RETRIEVAL_METRIC} \
                --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/main.py", line 10, in <module>
    from src.gpu_utils import GPUConfig
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


### Experiments on NYTimes to calculate F1 Score using Jaccard Similarity

In [24]:
THRESHOLD = [.1, .2, .4, .5, .6, .7, .85, .95]
threshold_str = ' '.join([str(t) for t in THRESHOLD])
RETRIEVAL_METRIC = 'f1'
DATASET = 'nytimes'
data_path = f'./data/{DATASET}_binary.npy'
SIMILARITY_SCORE = 'jaccard_similarity'

In [25]:
!python save_ground_truth.py --experiment 2 \
                             --data_path {data_path} \
                             --similarity_score {SIMILARITY_SCORE} \
                             --train_ratio .9 \
                             --seed 42 \
                             --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/save_ground_truth.py", line 15, in <module>
    from src import cosine_similarity, jaccard_similarity, inner_product
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


In [26]:
!python main.py --experiment 2 \
                --algo BinSketch BCS MinHash \
                --data_path {data_path} \
                --ground_truth_path ground_truth_exp2_{DATASET}_{SIMILARITY_SCORE}.json \
                --train_ratio .9 \
                --seed 42 \
                --threshold {threshold_str} \
                --similarity_score {SIMILARITY_SCORE} \
                --retrieval_metric {RETRIEVAL_METRIC} \
                --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/main.py", line 10, in <module>
    from src.gpu_utils import GPUConfig
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


### Experiments on KOS to calculate F1 Score using Cosine Similarity

In [27]:
THRESHOLD = [.1, .2, .4, .5, .6, .7, .85, .95]
threshold_str = ' '.join([str(t) for t in THRESHOLD])
RETRIEVAL_METRIC = 'f1'
DATASET = 'kos'
data_path = f'./data/{DATASET}_binary.npy'
SIMILARITY_SCORE = 'cosine_similarity'

In [28]:
!python save_ground_truth.py --experiment 2 \
                             --data_path {data_path} \
                             --similarity_score {SIMILARITY_SCORE} \
                             --train_ratio .9 \
                             --seed 42 \
                             --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/save_ground_truth.py", line 15, in <module>
    from src import cosine_similarity, jaccard_similarity, inner_product
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)


In [29]:
!python main.py --experiment 2 \
                --algo BinSketch BCS MinHash \
                --data_path {data_path} \
                --ground_truth_path ground_truth_exp2_{DATASET}_{SIMILARITY_SCORE}.json \
                --train_ratio .9 \
                --seed 42 \
                --threshold {threshold_str} \
                --similarity_score {SIMILARITY_SCORE} \
                --retrieval_metric {RETRIEVAL_METRIC} \
                --use_gpu

Traceback (most recent call last):
  File "/kaggle/working/binsketch-algorithm/main.py", line 10, in <module>
    from src.gpu_utils import GPUConfig
  File "/kaggle/working/binsketch-algorithm/src/__init__.py", line 3, in <module>
    from .metric import mse, minus_log_mse, precision, recall, f1_score
ImportError: cannot import name 'precision' from 'src.metric' (/kaggle/working/binsketch-algorithm/src/metric.py)
